## Base Case with Multiple Stock - Duality

In this nb, we solve the cutting optima problem with big order quantity to cut many specs at the same time with many stocks ( stock with same width and nearly same weight)
The condition/constraints as below: 
- `Generate Pattern` that fit in stock choice and sum each FG cut > lower bound demand
- `Filter Pattern` that keep the trim_cost <= 4%
- `Cut Pattern` that cut many stocks with filtered_pattern to minimize the stocks used

### 0. Helpers

In [1]:
from typing import Dict, Any

def check_finish_weight_per_stock(weight_s: float, width_s: float, finish: Dict[str, Dict[str, Any]]) -> Dict[str, bool]:
    """
    DOMAIN KNOWLEDGE:
    - As upper_demand is slightly/or significant higher than need_cut, 
    - We can check to pick stock that have weight per slice cut < upper_demand
    -  So that, at least, we can cut 1 slice of the FG with that kind of stock

    PICK STOCK WITH ALL/ALMOST TRUE VALUE (SUM TRUE LARGEST)
    """
    # MC WEIGHT PER UNIT
    wu = weight_s / width_s
    # dict of weight per slice FG on the chosen stock
    weight_slice_f = {f: finish[f]["width"]*wu for f in finish.keys()}
    # dict of upperbound demand FG on the chosen stock
    f_upper_demand = {f: finish[f]["upper_bound"] for f in finish.keys()} # 3 month forecast as default

    # dict of Yes/No 
    check_f ={f: weight_slice_f[f] < f_upper_demand[f] for f in finish.keys()}
    num_var_f = {f: 1 if weight_slice_f[f] < f_upper_demand[f] else 0 for f in finish.keys()}

    return check_f, num_var_f

def create_finish_demand_by_slice(stock, finish: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Convert demand in KGs to demand in slice on a specific stock
    """
    # MC WEIGHT PER UNIT
    wu = stock['weight'] / stock['width']
    # Dict of weight per slice FG on the chosen stock
    weight_slice_f = {f: finish[f]["width"]*wu for f in finish.keys()}

    dump_ls = {}
    for f, finish_info in finish.items():
         dump_ls[f] = {**finish_info
                            ,"upper_demand_slice": round(finish[f]["upper_bound"]/weight_slice_f[f])
                            , "demand_slice": round(finish[f]["need_cut"]/weight_slice_f[f])}
    
    # Filtering the dictionary to include only items with keys in keys_to_keep
    new_finish_list = {k: v for k, v in dump_ls.items() if v['upper_demand_slice'] > 0}

    return new_finish_list

def count_cut(data):
    # Initialize an empty dictionary for the total sums
    total_sums = {}
    
    # Loop through each dictionary in the list
    for entry in data:
        count = entry['count']
        cuts = entry['cuts']
        
        # Loop through each key in the cuts dictionary
        for key, value in cuts.items():
            # If the key is not already in the total_sums dictionary, initialize it to 0
            if key not in total_sums:
                total_sums[key] = 0
            # Add the product of count and value to the corresponding key in the total_sums dictionary
            total_sums[key] += count * value
    return total_sums

def count_weight(data):
    # Initialize an empty dictionary for the total sums
    total_sums = {}
    
    # Loop through each dictionary in the list
    for entry in data:
        count = entry['count']
        cuts = entry['cuts']
        
        # Loop through each key in the cuts dictionary
        for key, value in cuts.items():
            # If the key is not already in the total_sums dictionary, initialize it to 0
            if key not in total_sums:
                total_sums[key] = 0
            # Add the product of count and value to the corresponding key in the total_sums dictionary
            total_sums[key] += count * value
    return total_sums

In [2]:
def make_naive_patterns_by_width(stocks, finish, MIN_MARGIN):
    """
    Generates patterns of feasible cuts from stock width to meet specified finish widths.

    Parameters:
    stocks (dict): A dictionary where keys are stock identifiers and values are dictionaries
                   with key 'length' representing the length of each stock.

    finish (dict): A dictionary where keys are finish identifiers and values are dictionaries
                   with key 'length' representing the required finish lengths.

    Returns:
    patterns (list): A list of dictionaries, where each dictionary represents a pattern of cuts.
                   Each pattern dictionary contains 'stock' (the stock identifier) and 'cuts'
                   (a dictionary where keys are finish identifiers and the value is the number
                   of cuts from the stock for each finish).
                   
                   Naive pattern with maximum number of cuts of each Finished Goods
                   that is closet to the required need_cut
                   and SUM(widths of FG) smaller Mother Coil width
    """

    patterns = []
    for f in finish:
        feasible = False
        for s in stocks:
            # max number of f that fit on s
            num_cuts = int((stocks[s]["width"]-MIN_MARGIN) / finish[f]["width"])

            # make pattern and add to list of patterns
            if num_cuts > 0:
                feasible = True
                cuts_dict = {key: 0 for key in finish.keys()}
                cuts_dict[f] = num_cuts
                patterns.append({"stock": s, "cuts": cuts_dict})

        if not feasible:
            print(f"No feasible pattern was found for Stock {s} and FG {f}")

    return patterns

def make_naive_patterns(stocks, finish, MIN_MARGIN):
    """
    Generates patterns of feasible cuts from stock width to meet specified finish widths.

    Parameters:
    stocks (dict): A dictionary where keys are stock identifiers and values are dictionaries
                   with key 'length' representing the length of each stock.

    finish (dict): A dictionary where keys are finish identifiers and values are dictionaries
                   with key 'length' representing the required finish lengths.

    Returns:
    patterns (list): A list of dictionaries, where each dictionary represents a pattern of cuts.
                   Each pattern dictionary contains 'stock' (the stock identifier) and 'cuts'
                   (a dictionary where keys are finish identifiers and the value is the number
                   of cuts from the stock for each finish).
                   
                   Naive pattern with maximum number of cuts of each Finished Goods
                   that is closet to the required need_cut
                   and SUM(widths of FG) smaller Mother Coil width
    """

    patterns = []
    for f in finish:
        feasible = False
        for s in stocks:
            # max number of f that fit on s
            num_cuts_by_width = int((stocks[s]["width"]-MIN_MARGIN) / finish[f]["width"])
            # max number of f that satisfied the need cut WEIGHT BOUND
            num_cuts_by_weight = int((finish[f]["upper_bound"] * stocks[s]["width"] ) / (finish[f]["width"] * stocks[s]['weight']))
            # min of two max will satisfies both
            num_cuts = min(num_cuts_by_width, num_cuts_by_weight)

            # make pattern and add to list of patterns
            if num_cuts > 0:
                feasible = True
                cuts_dict = {key: 0 for key in finish.keys()}
                cuts_dict[f] = num_cuts
                patterns.append({"stock": s, "cuts": cuts_dict})

        if not feasible:
            print(f"No feasible pattern was found for Stock {s} and FG {f}")

    return patterns

def filter_high_fit_stock(stocks,finish):
    
    new_stock_list = {}
    max_f_fit = 0
    for stock_name, stock_info in stocks.items():
        _, num_var_f = check_finish_weight_per_stock(stock_info["weight"], 
                                                     stock_info["width"], finish)
        total_f_fit = sum(num_var_f.values())

        if total_f_fit > max_f_fit:
            #new_stock_list is reset 
            new_stock_list = {stock_name: {**stock_info, "total_f_fit": total_f_fit}} 
            max_f_fit = total_f_fit
        elif total_f_fit == max_f_fit:
            new_stock_list[stock_name] = {**stock_info, "total_f_fit": total_f_fit}
    
    return new_stock_list


### 1. Loading data

In [3]:
## PARAMETER 
#   DOMAIN KNOWLEDGE:
# - User usually overcut current order_quantity (30 -50%) default on model
# - Users want to cut all PO qty in one stock cut ( increase boundary to max 3 months forecast)
PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH440W-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 2
          ,"maker" : "POSCO-KR"
          ,"stock_ratio": { #get from app 
                    "limited": None
                    # "default": 2
                    # "user_setting": 4
                }
        #   ,"forecast_scenario": median
          }

MARGIN_DICT = { "HSC":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    },
    "NQS":
    {#save margin_dict in azure env, margin dictionary by warehouse
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
    }
}

# GET ALL PARAMS
MIN_MARGIN = MARGIN_DICT[PARAMS["warehouse"]][f"thickness_{PARAMS["thickness"]}"]["margin"]
print(f"MIN_MARGIN:{MIN_MARGIN}")

## PARAMETER
BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_KEY:{BOUND_KEY}, BOUND_VALUE:{BOUND_VALUE}")
LIMIT_BOUND_KEY, LIMIT_BOUND_VALUE =  "limited", None

MIN_MARGIN:8
BOUND_KEY:limited, BOUND_VALUE:None


### 2. SOLVER - with duality for pattern generation

In [4]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD, value

# CUT KNOWN SLICE PATTERNS
def cut_slice_patterns(finish, patterns):
    # Create a LP minimization problem
    prob = LpProblem("PatternCuttingProblem", LpMinimize)

    # Define variables
    x = {p: LpVariable(f"x_{p}", lowBound=0, cat='Integer') for p in range(len(patterns))}

    # Objective function: minimize total stock use
    prob += lpSum(x[p] for p in range(len(patterns))), "TotalCost"

    # Constraints: meet demand for each finished part
    for f in finish:
        prob += lpSum(patterns[p]['cuts'][f] * x[p] for p in range(len(patterns))) >= finish[f]['demand_slice'], f"DemandSlice{f}"

    # Solve the problem
    prob.solve()

    # Extract results
    solution = [x[p].varValue for p in range(len(patterns))]
    total_cost = sum(solution[p] for p in range(len(patterns)))

    return solution, total_cost

# NEW PATTERN - OBJECT FUNC KO WORK CHO WEIGHT
def new_pattern_problem(finish, width_s, ap_upper_bound, demand_duals, MIN_MARGIN):
    prob = LpProblem("NewPatternProblem", LpMaximize)

    # Decision variables - Pattern
    ap = {f: LpVariable(f"ap_{f}", 0, ap_upper_bound[f], cat="Integer") for f in finish.keys()}

    # Objective function
    # maximize marginal_cut:
    prob += lpSum(ap[f] * demand_duals[f] for f in finish.keys()), "MarginalCut"

    # Constraints - subject to stock_width - MIN MARGIN
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) <= width_s - MIN_MARGIN, "StockWidth_MinMargin"
    
    # Constraints - subject to trim loss 4% 
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) >= 0.96 * width_s , "StockWidth"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    marg_cost = value(prob.objective)
    pattern = {f: int(ap[f].varValue) for f in finish.keys()}
    return marg_cost, pattern

# DUALITY
def generate_pattern_dual(stocks, finish, patterns, MIN_MARGIN):
    prob = LpProblem("GeneratePatternDual", LpMinimize)

    # Sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    s = {p: patterns[p]["stock"] for p in range(len(patterns))}
    a = {(f, p): patterns[p]["cuts"][f] for p in P for f in F}
    demand_finish = {f: finish[f]["demand_slice"] for f in F}

    # Decision variables #relaxed integrality
    x = {p: LpVariable(f"x_{p}", 0, None, cat="Continuous") for p in P}

    # OBJECTIVE function minimize stock used:
    prob += lpSum(x[p] for p in P), "Cost"

    # Constraints
    for f in F:
        prob += lpSum(a[f, p] * x[p] for p in P) >= demand_finish[f], f"Demand_{f}"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    # Extract dual values
    dual_values = {f: prob.constraints[f"Demand_{f}"].pi for f in F}

    ap_upper_bound = {f: max([patterns[i]['cuts'][f] for i,_ in enumerate(patterns)]) for f in finish.keys()}
    demand_duals = {f: dual_values[f] for f in F}

    marginal_values = {}
    pattern = {}
    for s in stocks.keys():
        marginal_values[s], pattern[s] = new_pattern_problem(
            finish, stocks[s]["width"], ap_upper_bound, demand_duals, MIN_MARGIN
        )

    s = max(marginal_values, key=marginal_values.get)
    new_pattern = {"stock": s, "cuts": pattern[s]}
    return new_pattern

In [5]:
# NEW PATTERN - OBJECT FUNC KO WORK CHO WEIGHT
def new_pattern_problem_w(finish, width_s, ap_upper_bound, demand_duals, MIN_MARGIN):
    prob = LpProblem("NewPatternProblem", LpMaximize)

    # Decision variables - Pattern
    ap = {f: LpVariable(f"ap_{f}", 0, ap_upper_bound[f], cat="Integer") for f in finish.keys()}

    # Objective function
    # maximize marginal_cut:
    prob += lpSum(ap[f] * demand_duals[f] for f in finish.keys()), "MarginalCut"

    # Constraints - subject to stock_width - MIN MARGIN
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) <= width_s - MIN_MARGIN, "StockWidth_MinMargin"
    
    # Constraints - subject to trim loss 4% 
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) >= 0.96 * width_s , "StockWidth"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    marg_cost = value(prob.objective)
    pattern = {f: int(ap[f].varValue) for f in finish.keys()}
    return marg_cost, pattern

# DUALITY BY WEIGHT
def generate_pattern_dual_w(stocks, finish, patterns, MIN_MARGIN):
    
    # Defince prob
    prob = LpProblem("GeneratePatternDual", LpMinimize)
    # Sets
    F = list(finish.keys())
    P = list(range(len(patterns)))
    
    # Parameters
    s = {p: patterns[p]["stock"] for p in range(len(patterns))}
    c = {p: stocks[pattern['stock']]["weight"]/stocks[pattern['stock']]["width"] for p, pattern in enumerate(patterns)}
    a = {(f, p): patterns[p]["cuts"][f] for p in P for f in F}
    demand_finish = {f: finish[f]["need_cut"] for f in F}
    
    # Decision variables #relaxed integrality
    x = {p: LpVariable(f"x_{p}", 0, None, cat="Continuous") for p in P}
    # OBJECTIVE function minimize stock used:
    prob += lpSum(x[p] for p in P), "Cost"
    # Constraints
    for f in F:
        prob += lpSum(a[f, p] * finish[f]['width'] * c[p] * x[p] for p in P) >= demand_finish[f], f"Demand_{f}"
    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))
    
    # Extract dual values
    dual_values = {f: prob.constraints[f"Demand_{f}"].pi for f in F}
    ap_upper_bound = {f: max([p['cuts'][f] for p in patterns]) for f in finish.keys()}
    
    demand_duals = {f: dual_values[f] for f in F}

    marginal_values = {}
    pattern = {}
    for s in stocks.keys():
        # print(s)
        marginal_values[s], pattern[s] = new_pattern_problem(
            finish, stocks[s]["width"], ap_upper_bound, demand_duals, MIN_MARGIN
        )
    # print(marginal_values)
    s = max(marginal_values, key=marginal_values.get)
    new_pattern = {"stock": s, "cuts": pattern[s]}
    
    return new_pattern

In [6]:
# CUT KNOWN WEIGHT PATTERNS
def cut_weight_patterns(stocks, finish, patterns):
    # Parameters - unit weight
    c = {p: stocks[pattern['stock']]["weight"]/stocks[pattern['stock']]["width"] for p, pattern in enumerate(patterns)}

    # Create a LP minimization problem
    prob = LpProblem("PatternCuttingProblem", LpMinimize)

    # Define variables
    # x = {p: LpVariable(f"x_{p}", lowBound=0, cat='Integer') for p in range(len(patterns))}
    x = {p: LpVariable(f"x_{p}", 0, 1, cat='Integer') for p in range(len(patterns))}

    # Objective function: minimize total stock use
    prob += lpSum(x[p]* c[p] for p in range(len(patterns))), "TotalCost"

    # Constraints: meet demand for each finished part
    for f in finish:
        prob += lpSum(patterns[p]['cuts'][f] * finish[f]['width'] * x[p] * c[p] for p in range(len(patterns))) >= finish[f]['need_cut'], f"DemandWeight{f}"

    # Solve the problem
    prob.solve()

    # Extract results
    solution = [x[p].varValue for p in range(len(patterns))]
    total_cost = sum(1 * solution[p] for p in range(len(patterns)))

    return solution, total_cost

### 3. SOLVING

********


In [ ]:
# DOMAIN KNOWLEDGE:
# - FG with small spec - weight is hard to fit in a large weight stock
# function bi sensitive voi total f fit qua lon

high_stock_list = filter_high_fit_stock(stocks, finish)
len(high_stock_list)

In [ ]:
patterns = make_naive_patterns(high_stock_list, finish, MIN_MARGIN)
print(f"Total patterns: {len(patterns)}")

# Filter finish list having pattern # 0
upper_bound = {f: max([p['cuts'][f] for p in patterns]) for f in finish.keys()}
filtered_finish = {k: v for k, v in finish.items() if upper_bound[k] > 0}

In [ ]:
# Phase 1: Generate patterns using dual method
print("Phase 1: Pattern Generation", end=".")
new_pattern = generate_pattern_dual_w(high_stock_list, filtered_finish, patterns, MIN_MARGIN)
while new_pattern not in patterns:
    patterns.append(new_pattern)
    new_pattern = generate_pattern_dual_w(high_stock_list, filtered_finish, patterns, MIN_MARGIN)
print(end=".")
print()
print(f'Len naive/dual patterns : {len(patterns)}')
print()

In [ ]:
# Phrase 2: Filter Patterns having trim loss as requirements
print("Phase 2: Filter Patterns", end=":")
print()
unique_stocks = {pattern['stock'] for pattern in patterns}
filtered_trimloss_pattern = []
for s in unique_stocks:
    print(f'process {s}')
    width_s = high_stock_list[s]["width"] 
    for pattern in patterns:
        cuts_dict= pattern['cuts']
        trim_loss = width_s - sum([filtered_finish[f]["width"] * cuts_dict[f] for f in filtered_finish.keys()])
        trim_loss_pct = trim_loss/width_s * 100
        print(f"***** take pattern {pattern} *****")
        filtered_trimloss_pattern.append(pattern)
        print(f"trim loss: {trim_loss},trim loss percent:{trim_loss_pct}") 

********************
#### >> Create Pattern with mixed stocks - Stock with same WIDTH and ~ WEIGHT

Lowest - Highest 10%

In [ ]:
# TEST DATA - NORMAL CASE , NO DEFECT, UPPER BOUND DEMAND IS 50% PO QTY (TRY THIS CASE FIRST)
stocks = {'S438': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0, 'weight': 6180, 'qty': 1},
          'S442': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 6780,  'qty': 1},
          'S440': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 6820,  'qty': 1},
          'S147': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 7010,  'qty': 1},
          'S144': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 7390,  'qty': 1}
 }

finish = {'F88': {'width': 390.0, 'need_cut': 2036, 'upper_bound': 4265.795, 'fc1': 7432.65, 'fc2': 4930.2,  'fc3': 4706.1},
 'F87': {'width': 354.0,  'need_cut': 9642,  'upper_bound': 23399.847840000002,  'fc1': 45859.4928,  'fc2': 39600.1584,  'fc3': 36639.6624},
 'F86': {'width': 269.0,  'need_cut': 8500,  'upper_bound': 13470.692319999998,  'fc1': 16568.9744,  'fc2': 23670.539200000003,  'fc3': 20857.32},
 'F84': {'width': 215.0,  'need_cut': 4300,  'upper_bound': 5582.8426,  'fc1': 4276.142,  'fc2': 4074.3419999999996, 'fc3': 3594.0579999999995},
 'F91': {'width': 70.0,  'need_cut': 1200,  'upper_bound': 1801.9351199999999,  'fc1': 2006.4503999999997,  'fc2': 1732.5911999999996,  'fc3': 1603.0631999999998}}

total_needcut = sum(item['need_cut'] for item in finish.values())
print(f'Need cut {total_needcut}')

In [ ]:
## Pick Stock 
s = "S144" # LOWEST WEIGHT  - > FINALLY IT CAN BE OVER-CUT AND ACCEPT
choosen_stock_list = {k: v for k, v in stocks.items() if k==s}
print(f"Choose Stock: {choosen_stock_list}")

# Pick Finish List
new_finish_list = create_finish_demand_by_slice(stocks[s], finish)
sum_needcut = sum(item['need_cut'] for item in new_finish_list.values())
print(f"Sum need cut: {sum_needcut}") 

# Create Naive Patterns
patterns = make_naive_patterns(choosen_stock_list, new_finish_list, MIN_MARGIN)
print(f"Total Naive patterns: {len(patterns)}")

In [ ]:
# Phase 1: Generate patterns using dual method
print("Phase 1: Pattern Generation", end=".")
new_pattern = generate_pattern_dual(choosen_stock_list, new_finish_list, patterns, MIN_MARGIN)
while new_pattern not in patterns:
    patterns.append(new_pattern)
    new_pattern = generate_pattern_dual(choosen_stock_list, new_finish_list, patterns, MIN_MARGIN)
    print(end=".")
print()
print(f'Total naive/dual patterns :{len(patterns)}')
print()
# Phrase 2: Filter Patterns having trim loss as requirements
print("Phase 2: Filter Patterns", end=":")
print()
filtered_trimloss_pattern = []
width_s = choosen_stock_list[s]["width"] 
for pattern in patterns:
    cuts_dict= pattern['cuts']
    trim_loss = width_s - sum([finish[f]["width"] * cuts_dict[f] for f in new_finish_list.keys()])
    trim_loss_pct = trim_loss/width_s*100
    pattern.update({'trim_loss':trim_loss, "trim_loss_pct": round(trim_loss_pct,2)})
    if trim_loss_pct <= 4.00:
        print(f"***** take pattern {pattern} *****")
        filtered_trimloss_pattern.append(pattern)
        # print(f"trim loss: {trim_loss}, trim loss percent:{trim_loss_pct}")

# Phrase 3: Cut Pattern to map all pattern to demand requirements
print()
print("Phase 3: Cut Patterns", end=".")
print()

In [ ]:
solution, total_cost = cut_slice_patterns(new_finish_list, filtered_trimloss_pattern)
# Evaluation - Calculate overcut- compared to need cut
solution_list = []
for i, count in enumerate(solution):
    if count > 0:
        cut_pattern = filtered_trimloss_pattern[i]['cuts']
        print(f"Take ({int(count)}) pattern: {cut_pattern}")
        sol = {"count":count,"cuts":cut_pattern}
        solution_list.append(sol)

In [ ]:
total_sums = count_cut(solution_list)
# MC WEIGHT PER UNIT
wu = stocks[s]['weight']/stocks[s]['width']
over_cut = {}
# Overcut cutweight (slice * fg_width * wu) - need_cut 
for key in total_sums.keys():
    over_cut[key] = total_sums[key] * finish[key]['width'] * wu - finish[key]['need_cut']

over_cut

*****
#### >> Create Pattern with mixed stocks - Stock with same WIDTH diff weight

In [209]:
# TEST DATA - NORMAL CASE , NO DEFECT, UPPER BOUND DEMAND IS 50% PO QTY (TRY THIS CASE FIRST)
# stocks = {'S438': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 4180, 'qty': 1},
#           'S442': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 5780,  'qty': 1},
#           'S440': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 6820,  'qty': 1},
#           'S147': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 7010,  'qty': 1},
#           'S144': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 7390,  'qty': 1},
#           'S148': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 7390,  'qty': 1},
#           'S145': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 7400,  'qty': 1},
#           'S146': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 8440,  'qty': 1}
#  }

finish = {'F88': {'width': 390.0, 'need_cut': 2036, 'upper_bound': 4265.795,'fc1': 7432.65,  'fc2': 4930.2,  'fc3': 4706.1},
        'F87': {'width': 354.0,  'need_cut': 9642,  'upper_bound': 23399.847840000002,  'fc1': 45859.4928,  'fc2': 39600.1584,  'fc3': 36639.6624},
        'F86': {'width': 269.0,  'need_cut': 8500,  'upper_bound': 13470.692319999998,  'fc1': 16568.9744,  'fc2': 23670.539200000003,  'fc3': 20857.32},
        'F84': {'width': 215.0,  'need_cut': 4300,  'upper_bound': 5582.8426,  'fc1': 4276.142,  'fc2': 4074.3419999999996, 'fc3': 3594.0579999999995},
        'F91': {'width': 70.0,  'need_cut': 200,  'upper_bound': 301.9351199999999,  'fc1': 1006.4503999999997,  'fc2': 1732.5911999999996,  'fc3': 1603.0631999999998}
        }

# DOMAIN KNOWLEDGE:
# - FG with small spec - weight is hard to fit in a large weight stock
# function bi sensitive voi total f fit qua lon


In [210]:
total_needcut = sum(item['need_cut'] for item in finish.values())
print(f'Need cut {total_needcut}')

# list of stock that can fit as many stock as possible
high_stock_list = filter_high_fit_stock(stocks, finish)
print(f"len(high_stock_list): {len(high_stock_list)}")

# Create Naive Patterns
nai_patterns = make_naive_patterns_by_width(high_stock_list, finish, MIN_MARGIN)
print(f"Total Naive patterns: {len(nai_patterns)}")

Need cut 24678
len(high_stock_list): 2
Total Naive patterns: 10


In [211]:
s = "S442" # MID WEIGHT  - > FINALLY IT CAN BE OVER-CUT AND ACCEPT
choosen_stock_list = {k: v for k, v in stocks.items() if k==s}
print(f"Choose Stock: {choosen_stock_list}")

new_finish_list = create_finish_demand_by_slice(stocks[s], finish)

Choose Stock: {'S442': {'receiving_date': '2022/07/18', 'status': 'M:RAW MATERIAL', 'width': 1219.0, 'weight': 4780, 'qty': 1}}


In [212]:
# Phase 1: Generate patterns using dual method
print("Phase 1: Pattern Generation", end=".")
dual_pat = []
patterns = nai_patterns
new_pattern = generate_pattern_dual(choosen_stock_list, new_finish_list, patterns, MIN_MARGIN)
while new_pattern not in patterns:
    patterns.append(new_pattern)
    dual_pat.append(new_pattern)
    new_pattern = generate_pattern_dual(choosen_stock_list, new_finish_list, patterns, MIN_MARGIN)
    print(end=".")
print()
print(f'Total naive/dual patterns :{len(patterns)}')
print()
# Phrase 2: Filter Patterns having trim loss as requirements
print("Phase 2: Filter Patterns", end=":")
print()
filtered_trimloss_pattern = []
width_s = choosen_stock_list[s]["width"] 

mul_dual_cut = []
for pattern in dual_pat:
    cuts_dict= pattern['cuts']
    for s in stocks.keys():
        new_pat = {"stock":s, "cuts":cuts_dict}
        mul_dual_cut.append(new_pat)

sum_patterns = nai_patterns + mul_dual_cut

for pattern in sum_patterns:
    cuts_dict= pattern['cuts']
    trim_loss = width_s - sum([finish[f]["width"] * cuts_dict[f] for f in new_finish_list.keys()])
    trim_loss_pct = trim_loss/ width_s*100
    pattern.update({'trim_loss':trim_loss, "trim_loss_pct": round(trim_loss_pct,2)})
    if trim_loss_pct <= 4.00: 
        # filter naive pattern appended
        # print(f"***** take pattern {pattern} *****")
        filtered_trimloss_pattern.append(pattern)
        # print(f"trim loss: {trim_loss}, trim loss percent:{round(trim_loss_pct,2)}")
print(f"filteredPattern:{len(filtered_trimloss_pattern)}")

# Phrase 3: Cut Pattern to map all pattern to demand requirements
print()
print("Phase 3: Cut Patterns", end=".")
print()

Phase 1: Pattern Generation.........
Total naive/dual patterns :18

Phase 2: Filter Patterns:
filteredPattern:74

Phase 3: Cut Patterns.


In [213]:
solution, total_cost = cut_weight_patterns(high_stock_list, new_finish_list, filtered_trimloss_pattern)
# Evaluation - Calculate overcut- compared to need cut
solution_list = []
for i, count in enumerate(solution):
    if count > 0:
        cut_pattern = filtered_trimloss_pattern[i]['cuts']
        print(f"Take ({int(count)}) stock: {filtered_trimloss_pattern[i]['stock']}, pattern: {cut_pattern}, trim loss: {filtered_trimloss_pattern[i]['trim_loss_pct']}")
        sol = {"count":count,"stock":filtered_trimloss_pattern[i]['stock'],"cuts":cut_pattern}
        solution_list.append(sol)

KeyError: 'S440'

###  Now try combination that can solve at once large qty order
#### >> Create Pattern with mixed stocks - NO.1 DYNAMIC - Stock with different weight and width

In [214]:
stocks = {'S438': {'receiving_date': '2022/07/18',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 6180,
  'qty': 1},
 'S442': {'receiving_date': '2022/07/18',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 6780,
  'qty': 1},
 'S440': {'receiving_date': '2022/07/18',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 6820,
  'qty': 1},
 'S147': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7010,
  'qty': 1},
 'S144': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7390,
  'qty': 1},
 'S148': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7390,
  'qty': 1},
 'S145': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7400,
  'qty': 1},
 'S146': {'receiving_date': '2022/05/19',
  'status': 'M:RAW MATERIAL',
  'width': 1219.0,
  'weight': 7440,
  'qty': 1}}

In [215]:
# stocks = {
#     'S438': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 5180, 'qty': 1},
#         'S442': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 4780,  'qty': 1},
#         'S440': {'receiving_date': '2022/07/18',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 5820,  'qty': 1},
#         'S147': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1080.0,  'weight': 6010,  'qty': 1},
#         'S144': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 7390,  'qty': 1},
#         'S148': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1223.0,  'weight': 7390,  'qty': 1},
#         'S145': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1219.0,  'weight': 7400,  'qty': 1},
#         'S146': {'receiving_date': '2022/05/19',  'status': 'M:RAW MATERIAL',  'width': 1080.0,  'weight': 8440,  'qty': 1}
#         }

finish = {'F88': {'width': 390.0, 'need_cut': 2036, 'upper_bound': 2265.795,'fc1': 2432.65,  'fc2': 2930.2,  'fc3': 2706.1},
        'F87': {'width': 354.0,  'need_cut': 9642,  'upper_bound': 13399.847840000002,  'fc1': 15859.4928,  'fc2': 19600.1584,  'fc3': 16639.6624},
        'F86': {'width': 269.0,  'need_cut': 8500,  'upper_bound': 13470.692319999998,  'fc1': 16568.9744,  'fc2': 13670.539200000003,  'fc3': 10857.32},
        'F84': {'width': 215.0,  'need_cut': 4300,  'upper_bound': 5582.8426,  'fc1': 4276.142,  'fc2': 4074.3419999999996, 'fc3': 3594.0579999999995},
        'F91': {'width': 70.0,  'need_cut': 100,  'upper_bound': 250,  'fc1': 1006.4503999999997,  'fc2': 1732.5911999999996,  'fc3': 1603.0631999999998}
        }

total_needcut = sum(item['need_cut'] for item in finish.values())
print(f'Need cut {total_needcut}')

Need cut 24578


In [216]:
# Create Naive Patterns
nai_patterns = make_naive_patterns(stocks, finish, MIN_MARGIN)
print(f"Total Naive patterns: {len(nai_patterns)}")\

def create_finish_demand_by_slice_fr_nai_pattern(patterns, finish: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Convert demand in KGs to demand in slice on a specific stock
    """

    dump_ls = {}
    for f, finish_info in finish.items():
        try:
            non_zero_min = min([patterns[i]['cuts'][f] for i, _ in enumerate(patterns) if patterns[i]['cuts'][f] != 0])
        except ValueError:
            non_zero_min = 0
        dump_ls[f] = {**finish_info
                            ,"upper_demand_slice": max([patterns[i]['cuts'][f] for i,_ in enumerate(patterns)])
                            ,"demand_slice": non_zero_min }
    
    # Filtering the dictionary to include only items with keys in keys_to_keep
    new_finish_list = {k: v for k, v in dump_ls.items() if v['upper_demand_slice'] > 0}

    return new_finish_list

new_finish_list = create_finish_demand_by_slice_fr_nai_pattern(nai_patterns,finish)
len(new_finish_list)

No feasible pattern was found for Stock S146 and FG F91
Total Naive patterns: 28


4

In [217]:
# NEW PATTERN - OBJECT FUNC KO WORK CHO WEIGHT
def new_pattern_problem(finish, width_s, ap_upper_bound, demand_duals, MIN_MARGIN):
    prob = LpProblem("NewPatternProblem", LpMaximize)

    # Decision variables - Pattern
    ap = {f: LpVariable(f"ap_{f}", 0, ap_upper_bound[f], cat="Integer") for f in finish.keys()}

    # Objective function
    # maximize marginal_cut:
    prob += lpSum(ap[f] * demand_duals[f] for f in finish.keys()), "MarginalCut"

    # Constraints - subject to stock_width - MIN MARGIN
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) <= width_s - MIN_MARGIN, "StockWidth_MinMargin"
    
    # Constraints - subject to trim loss 4% 
    prob += lpSum(ap[f] * finish[f]["width"] for f in finish.keys()) >= 0.96 * width_s , "StockWidth"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    marg_cost = value(prob.objective)
    pattern = {f: int(ap[f].varValue) for f in finish.keys()}
    return marg_cost, pattern

# DUALITY
def generate_pattern_dual(stocks, finish, patterns, MIN_MARGIN):
    prob = LpProblem("GeneratePatternDual", LpMinimize)

    # Sets
    F = list(finish.keys())
    P = list(range(len(patterns)))

    # Parameters
    s = {p: patterns[p]["stock"] for p in range(len(patterns))}
    a = {(f, p): patterns[p]["cuts"][f] for p in P for f in F}
    demand_finish = {f: finish[f]["demand_slice"] for f in F}

    # Decision variables #relaxed integrality
    x = {p: LpVariable(f"x_{p}", 0, None, cat="Continuous") for p in P}

    # OBJECTIVE function minimize stock used:
    prob += lpSum(x[p] for p in P), "Cost"

    # Constraints
    for f in F:
        prob += lpSum(a[f, p] * x[p] for p in P) >= demand_finish[f], f"Demand_{f}"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=False, options=['--solver', 'highs']))

    # Extract dual values
    dual_values = {f: prob.constraints[f"Demand_{f}"].pi for f in F}

    ap_upper_bound = {f: max([patterns[i]['cuts'][f] for i,_ in enumerate(patterns)]) for f in finish.keys()}
    demand_duals = {f: dual_values[f] for f in F}

    marginal_values = {}
    pattern = {}
    for s in stocks.keys():
        marginal_values[s], pattern[s] = new_pattern_problem(
            finish, stocks[s]["width"], ap_upper_bound, demand_duals, MIN_MARGIN
        )

    s = max(marginal_values, key=marginal_values.get)
    new_pattern = {"stock": s, "cuts": pattern[s]}
    return new_pattern

In [218]:
# Phase 1: Generate patterns using dual method
print("Phase 1: Pattern Generation", end=".")
dual_pat = []
patterns = nai_patterns
new_pattern = generate_pattern_dual(stocks, new_finish_list, patterns, MIN_MARGIN)
while new_pattern not in patterns:
    patterns.append(new_pattern)
    dual_pat.append(new_pattern)
    new_pattern = generate_pattern_dual(stocks, new_finish_list, patterns, MIN_MARGIN)
    print(end=".")
print()
print(f'Total naive/dual patterns :{len(patterns)}')
print()

Phase 1: Pattern Generation.

....
Total naive/dual patterns :32



In [219]:
# Phrase 2: Filter Patterns having trim loss as requirements
print("Phase 2: Filter Patterns", end=":")
print()
filtered_trimloss_pattern = []

for pattern in patterns:
    cuts_dict= pattern['cuts']
    width_s = stocks[pattern['stock']]['width']
    trim_loss = width_s - sum([finish[f]["width"] * cuts_dict[f] for f in new_finish_list.keys()])
    trim_loss_pct = trim_loss/ width_s*100
    pattern.update({'trim_loss':trim_loss, "trim_loss_pct": round(trim_loss_pct,2)})
    if trim_loss_pct <= 4.00: 
        # filter naive pattern appended
        # print(f"***** take pattern {pattern} *****")
        filtered_trimloss_pattern.append(pattern)
        # print(f"trim loss: {trim_loss}, trim loss percent:{round(trim_loss_pct,2)}")
print(f"filteredPattern:{len(filtered_trimloss_pattern)}")

# Phrase 3: Cut Pattern to map all pattern to demand requirements
print()
print("Phase 3: Cut Patterns", end=".")
print()

Phase 2: Filter Patterns:
filteredPattern:4

Phase 3: Cut Patterns.


In [220]:
# CUT KNOWN WEIGHT PATTERNS
def cut_weight_patterns(stocks, finish, patterns):
    # Parameters - unit weight
    c = {p: stocks[pattern['stock']]["weight"]/stocks[pattern['stock']]["width"] for p, pattern in enumerate(patterns)}

    # Create a LP minimization problem
    prob = LpProblem("PatternCuttingProblem", LpMinimize)

    # Define variables
    x = {p: LpVariable(f"x_{p}", lowBound=0, cat='Integer') for p in range(len(patterns))}
    # x = {p: LpVariable(f"x_{p}", 0, 1, cat='Integer') for p in range(len(patterns))}

    # Objective function: minimize total stock use
    prob += lpSum(x[p]* c[p] for p in range(len(patterns))), "TotalCost"

    # Constraints: meet demand for each finished part
    for f in finish:
        prob += lpSum(patterns[p]['cuts'][f] * finish[f]['width'] * x[p] * c[p] for p in range(len(patterns))) >= finish[f]['need_cut'], f"DemandWeight{f}"

    # Solve the problem
    prob.solve()

    # Extract results
    solution = [x[p].varValue for p in range(len(patterns))]
    total_cost = sum(1 * solution[p] for p in range(len(patterns)))

    return solution, total_cost

In [221]:
solution, total_cost = cut_weight_patterns(stocks, new_finish_list, filtered_trimloss_pattern)
# Evaluation - Calculate overcut- compared to need cut
solution_list = []
for i, count in enumerate(solution):
    if count > 0.5:
        cut_pattern = filtered_trimloss_pattern[i]['cuts']
        print(f"Take ({int(count)}) stock: {filtered_trimloss_pattern[i]['stock']}, pattern: {cut_pattern}, trim loss: {filtered_trimloss_pattern[i]['trim_loss_pct']}")
        sol = {"count": int(count),"stock": filtered_trimloss_pattern[i]['stock'],"cuts":cut_pattern}
        solution_list.append(sol)

Take (2) stock: S438, pattern: {'F88': 1, 'F87': 0, 'F86': 3, 'F84': 0}, trim loss: 1.8
Take (4) stock: S438, pattern: {'F88': 0, 'F87': 2, 'F86': 1, 'F84': 1}, trim loss: 2.21


In [222]:
solution_list

[{'count': 2,
  'stock': 'S438',
  'cuts': {'F88': 1, 'F87': 0, 'F86': 3, 'F84': 0}},
 {'count': 4,
  'stock': 'S438',
  'cuts': {'F88': 0, 'F87': 2, 'F86': 1, 'F84': 1}}]

In [223]:
# cut_in_weight = 
for i, sol in enumerate(solution_list):
    s = solution_list[i]['stock']
    cuts_dict = solution_list[i]['cuts']
    weight_dict = {f: round(cuts_dict[f]*finish[f]['width']*stocks[s]['weight']/stocks[s]['width'],2) for f in cuts_dict.keys()}
    print(weight_dict)
    solution_list[i] = {**sol,
                        "cut_w": weight_dict}
    # solution_list[i]['cut_w'] = {f: cuts_dict[f]*stocks[s]['weight']/stocks[s]['width'] for f in cuts_dict.keys}


{'F88': 1977.19, 'F87': 0.0, 'F86': 4091.27, 'F84': 0.0}
{'F88': 0.0, 'F87': 3589.37, 'F86': 1363.76, 'F84': 1089.99}


In [224]:
def count_weight(data):
    # Initialize an empty dictionary for the total sums
    total_sums = {}
    
    # Loop through each dictionary in the list
    for entry in data:
        count = entry['count']
        cuts = entry['cut_w']
        
        # Loop through each key in the cuts dictionary
        for key, value in cuts.items():
            # If the key is not already in the total_sums dictionary, initialize it to 0
            if key not in total_sums:
                total_sums[key] = 0
            # Add the product of count and value to the corresponding key in the total_sums dictionary
            total_sums[key] += count * value
    return total_sums

In [225]:
total_sums = count_weight(solution_list)
total_sums

{'F88': 3954.38, 'F87': 14357.48, 'F86': 13637.58, 'F84': 4359.96}

In [226]:
# MC WEIGHT PER UNIT
over_cut = {}
over_cut_ratio = {}
# Overcut cutweight (slice * fg_width * wu) - need_cut 
for key in total_sums.keys():
    over_cut[key] = total_sums[key] - finish[key]['need_cut']
    over_cut_ratio[key] = over_cut[key]/finish[key]['fc1']


over_cut_ratio

{'F88': 0.7885967977308697,
 'F87': 0.2973285501286649,
 'F86': 0.3100722999487524,
 'F84': 0.01402198523809547}